In [67]:
import pandas as pd
import numpy as np
import typing as t

In [68]:
table = pd.read_csv('table.csv')
table

,State,NextState,Accept,Return,Error,Stack,M
0,100,1000.0,0,0,1,0,type id const return } eof
1,110,1100.0,0,0,0,0,type id const return
2,111,1110.0,0,0,1,0,} eof
3,120,1200.0,0,0,0,0,type
4,121,1210.0,0,0,0,0,id
...,...,...,...,...,...,...,...
149,3001,310.0,0,0,1,0,","
150,3010,NaN,0,1,1,0,)
151,3100,3101.0,1,0,1,0,","
152,3101,3102.0,1,0,1,0,number


In [69]:
np.unique(table[['Accept', 'Return', 'Error', 'Stack']].values, axis=0)

array([[0, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 1],
       [0, 1, 1, 0],
       [1, 0, 1, 0],
       [1, 1, 1, 0]])

In [70]:
class StateStack:
    def __init__(self) -> None:
        self.stack = []

    def push(self, value: int) -> None:
        self.stack.append(value)

    def pop(self) -> int:
        return self.stack.pop(-1)
    
    def is_empty(self) -> bool:
        return len(self.stack) == 0

In [71]:
class Lexer:
    def __init__(self, guiding_symbols: t.Dict[int, t.List[str]]) -> None:
        self.token_sequence = None
        self.guiding_symbols = guiding_symbols

    def get_current_token(self) -> str:
        return self.token_sequence[0]
    
    def set_token_sequence(self, token_sequence: str) -> None:
        self.token_sequence = token_sequence
    
    def check(self, current_state_number: int) -> None:
        assert self.get_current_token() in self.guiding_symbols[current_state_number]

    def accept(self) -> None:
        self.token_sequence = self.token_sequence[1:]

In [72]:
from abc import ABC, abstractmethod
from __future__ import annotations


class State(ABC):
    @abstractmethod
    def next(self) -> int:
        pass

In [73]:
class ZeroState(State):
    def __init__(self, state_number: int, lexer: Lexer, stack: StateStack, next_state: int) -> None:
        self._state_number = state_number
        self._lexer = lexer
        self._stack = stack
        self._next_state = next_state

        self.is_final = False

    def next(self) -> int:
        try:
            self._lexer.check(self._state_number)
            return self._next_state
        except:
            return self._state_number + 1

In [74]:
class ErrorState(State):
    def __init__(self, state_number: int, lexer: Lexer, stack: StateStack, next_state: int) -> None:
        self._state_number = state_number
        self._lexer = lexer
        self._stack = stack
        self._next_state = next_state

        self.is_final = False

    def next(self) -> int:
        try:
            self._lexer.check(self._state_number)
            return self._next_state
        except:
            raise Exception()

In [75]:
class ErrorStackState(State):
    def __init__(self, state_number: int, lexer: Lexer, stack: StateStack, next_state: int) -> None:
        self._state_number = state_number
        self._lexer = lexer
        self._stack = stack
        self._next_state = next_state

        self.is_final = False

    def next(self) -> int:
        try:
            self._lexer.check(self._state_number)
            self._stack.push(self._state_number + 1)
            return self._next_state
        except:
            raise Exception()

In [76]:
class ReturnErrorState(State):
    def __init__(self, state_number: int, lexer: Lexer, stack: StateStack, next_state: int) -> None:
        self._state_number = state_number
        self._lexer = lexer
        self._stack = stack
        self._next_state = next_state

        self.is_final = True

    def next(self) -> int:
        try:
            self._lexer.check(self._state_number)
            return self._stack.pop()
        except:
            raise Exception()

In [77]:
class AcceptErrorState(State):
    def __init__(self, state_number: int, lexer: Lexer, stack: StateStack, next_state: int) -> None:
        self._state_number = state_number
        self._lexer = lexer
        self._stack = stack
        self._next_state = next_state

        self.is_final = False

    def next(self) -> int:
        try:
            self._lexer.check(self._state_number)
            self._lexer.accept()
            return self._next_state
        except:
            raise Exception()

In [78]:
class AcceptReturnErrorState(State):
    def __init__(self, state_number: int, lexer: Lexer, stack: StateStack, next_state: int) -> None:
        self._state_number = state_number
        self._lexer = lexer
        self._stack = stack
        self._next_state = next_state

        self.is_final = True

    def next(self) -> int:
        try:
            self._lexer.check(self._state_number)
            self._lexer.accept()
            return self._stack.pop()
        except:
            raise Exception()

In [79]:
class LL1Parser:
    def __init__(self, rules: pd.DataFrame) -> None:
        self._lexer = Lexer(self.__get_guiding_symbols(rules))
        self._stack = StateStack()
        self._state_dict = self.__get_state_dict(rules, self._lexer, self._stack)

    def __get_guiding_symbols(self, rules: pd.DataFrame) -> t.Dict[int, t.List[str]]:
        return {
            row['State']: row['M'].split() for _, row in rules.iterrows()
        }

    def __get_state_dict(self, rules: pd.DataFrame, lexer: Lexer, stack: StateStack) -> t.Dict[int, State]:
        row2class = {
            (0, 0, 0, 0): ZeroState,
            (0, 0, 1, 0): ErrorState,
            (0, 0, 1, 1): ErrorStackState,
            (0, 1, 1, 0): ReturnErrorState,
            (1, 0, 1, 0): AcceptErrorState,
            (1, 1, 1, 0): AcceptReturnErrorState,
        }
        return {
            row['State']: row2class[tuple(row[['Accept', 'Return', 'Error', 'Stack']])](
                row['State'], 
                lexer, 
                stack, 
                row['NextState']
            ) for _, row in rules.iterrows()
        }
    
    def analyze(self, word: str, start_state_id: int = 100, eof_symbol: str = 'eof') -> bool:
        token_sequence = word.split()
        token_sequence.append(eof_symbol)
        self._lexer.set_token_sequence(token_sequence)

        while not (self._stack.is_empty() and self._lexer.get_current_token() == eof_symbol and self._state_dict[start_state_id].is_final):
            try:
                start_state_id = self._state_dict[start_state_id].next()
            except:
                return False
        return True
        

In [80]:
ll1 = LL1Parser(table)

In [ ]:
import re


example = """
type id ;
type id = ( number + number ) / ( number - number * ( id > number ) ) ;
type id = true && false ;
type id = id || true ;
type id = ! id ;
type id = ! true ;
id = id >= id ;
const type id = number ;
type id ( type id , type id ) {
    type id = id + number ;
    return id + id ;
} ;
"""

example = re.sub('\n', ' ', re.sub('\t', '', example))

In [82]:
ll1.analyze(example)

True